In [1]:
# 기본
import torch
import torch.nn as nn
transformer_model = nn.Transformer()
src = torch.rand((10, 32, 512))
tgt = torch.rand((20, 32, 512))
out = transformer_model(src, tgt)
print(out.size())

torch.Size([20, 32, 512])


In [2]:
# 만든거
from chatbot.model import CustomEncoder, CustomDecoder
import torch
src = torch.rand((20, 32, 512))
tgt = torch.rand((20, 32, 512))
enc = CustomEncoder()
dec = CustomDecoder()
memory = enc(src)
print("memory.size() :", memory.size()) #N,d_memory
dec_out = dec(tgt, memory)
print("dec_out.size() :", dec_out.size())

transformer = nn.Transformer(custom_encoder=enc, custom_decoder=dec)
transformer_out = transformer(src, tgt)
print("transformer_out.size() : ",transformer_out.size())

memory.size() : torch.Size([32, 2048])
dec_out.size() : torch.Size([20, 32, 512])
transformer_out.size() :  torch.Size([20, 32, 512])


            - src: :math:`(S, N, E)`.
            - tgt: :math:`(T, N, E)`.
            - src_mask: :math:`(S, S)`.
            - tgt_mask: :math:`(T, T)`.
            - memory_mask: :math:`(T, S)`.
            - src_key_padding_mask: :math:`(N, S)`.
            - tgt_key_padding_mask: :math:`(N, T)`.
            - memory_key_padding_mask: :math:`(N, S)`.
S : source sequence length  
T : target sequence length  
N : batch size  
E : feature number  
            

# (S,N)[0]
![title](https://pytorch.org/tutorials/_images/transformer_input_target.png) 

In [3]:
import io
import torch
from torchtext.utils import download_from_url, extract_archive
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

url = 'https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-2-v1.zip'
test_filepath, valid_filepath, train_filepath = extract_archive(download_from_url(url))
tokenizer = get_tokenizer('basic_english')
vocab = build_vocab_from_iterator(map(tokenizer,
                                      iter(io.open(train_filepath,
                                                   encoding="utf8"))))

def data_process(raw_text_iter):
    data = [torch.tensor([vocab[token] for token in tokenizer(item)],
                         dtype=torch.long) for item in raw_text_iter]
    return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))

train_data = data_process(iter(io.open(train_filepath, encoding="utf8")))
val_data = data_process(iter(io.open(valid_filepath, encoding="utf8")))
test_data = data_process(iter(io.open(test_filepath, encoding="utf8")))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def batchify(data, bsz):
    # Divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)

batch_size = 20
eval_batch_size = 10
train_data = batchify(train_data, batch_size)
val_data = batchify(val_data, eval_batch_size)
test_data = batchify(test_data, eval_batch_size)

print(train_data.size())

bptt = 35
def get_batch(source, i):
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    #target = source[i+1:i+1+seq_len].reshape(-1)
    target = source[i+1:i+1+seq_len]
    return data, target

data, targets = get_batch(train_data, 0)
print(data.size(), targets.size())
print(data.T[0],'\n', targets.reshape(data.size()).T[0])

36718lines [00:01, 26204.43lines/s]


torch.Size([102499, 20])
torch.Size([35, 20]) torch.Size([35, 20])
tensor([   10,  3850,  3870,   882,    10, 20001,    84,  3850,    89,     0,
         3870,    22,   781, 28781,     3,  6183,     4,  3850,     5,     2,
         5024,    89,    21,     3,  1838,  1019,     8,    15,  3850,  3870,
          882,   630,   977,     3,    24], device='cuda:0') 
 tensor([ 3850,  3870,   882,    10, 20001,    84,  3850,    89,     0,  3870,
           22,   781, 28781,     3,  6183,     4,  3850,     5,     2,  5024,
           89,    21,     3,  1838,  1019,     8,    15,  3850,  3870,   882,
          630,   977,     3,    24,     9], device='cuda:0')


In [4]:
ntokens = len(vocab.stoi) # the size of vocabulary
emsize = 200 # embedding dimension
nhid = 200 # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2 # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2 # the number of heads in the multiheadattention models
dropout = 0.2 # the dropout value
#model = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout).to(device)
d_memory = 200

In [5]:
from chatbot import Chatbot
model = Chatbot(ntokens, emsize, nhead, nhid, nlayers, d_memory, dropout).to(device)

In [6]:
import torch. nn as nn
criterion = nn.CrossEntropyLoss()
lr = 5.0 # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

import math
import time
def train():
    model.train() # Turn on the train mode
    total_loss = 0.
    start_time = time.time()
    src_mask = model.generate_square_subsequent_mask(bptt).to(device)
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        optimizer.zero_grad()
        if data.size(0) != bptt:
            src_mask = model.generate_square_subsequent_mask(data.size(0)).to(device)
        output = model(data, targets, src_mask)
        #print("output.size() :", output.size())
        #print("output.view(-1, ntokens) :", output.view(-1, ntokens).size())
        #print("targets.size() :", targets.size())
        #print("targets.reshape(-1).size() :", targets.reshape(-1).size())
        
        #loss = criterion(output.view(-1, ntokens), targets)
        loss = criterion(output.view(-1, ntokens), targets.reshape(-1))
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        log_interval = 200
        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:02.2f} | ms/batch {:5.2f} | '
                  'loss {:5.2f} | ppl {:8.2f}'.format(
                    epoch, batch, len(train_data) // bptt, scheduler.get_lr()[0],
                    elapsed * 1000 / log_interval,
                    cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()

def evaluate(eval_model, data_source):
    eval_model.eval() # Turn on the evaluation mode
    total_loss = 0.
    src_mask = model.generate_square_subsequent_mask(bptt).to(device)
    with torch.no_grad():
        for i in range(0, data_source.size(0) - 1, bptt):
            data, targets = get_batch(data_source, i)
            if data.size(0) != bptt:
                src_mask = model.generate_square_subsequent_mask(data.size(0)).to(device)
            output = eval_model(data, targets, src_mask)
            output_flat = output.view(-1, ntokens)
            total_loss += len(data) * criterion(output_flat, targets.reshape(-1)).item()
    return total_loss / (len(data_source) - 1)

In [7]:
best_val_loss = float("inf")
epochs = 2 # The number of epochs
best_model = None

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train()
    val_loss = evaluate(model, val_data)
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
          'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                     val_loss, math.exp(val_loss)))
    print('-' * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model

    scheduler.step()

/home/insublee/anaconda3/envs/chatbot/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:369: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


| epoch   1 |   200/ 2928 batches | lr 5.00 | ms/batch 30.96 | loss  2.59 | ppl    13.28
| epoch   1 |   400/ 2928 batches | lr 5.00 | ms/batch 30.36 | loss  0.64 | ppl     1.90
| epoch   1 |   600/ 2928 batches | lr 5.00 | ms/batch 29.70 | loss  0.39 | ppl     1.48
| epoch   1 |   800/ 2928 batches | lr 5.00 | ms/batch 29.62 | loss  0.28 | ppl     1.32
| epoch   1 |  1000/ 2928 batches | lr 5.00 | ms/batch 29.64 | loss  0.19 | ppl     1.21
| epoch   1 |  1200/ 2928 batches | lr 5.00 | ms/batch 29.65 | loss  0.17 | ppl     1.18
| epoch   1 |  1400/ 2928 batches | lr 5.00 | ms/batch 29.58 | loss  0.16 | ppl     1.18
| epoch   1 |  1600/ 2928 batches | lr 5.00 | ms/batch 29.61 | loss  0.13 | ppl     1.14
| epoch   1 |  1800/ 2928 batches | lr 5.00 | ms/batch 29.65 | loss  0.10 | ppl     1.10
| epoch   1 |  2000/ 2928 batches | lr 5.00 | ms/batch 29.63 | loss  0.08 | ppl     1.08
| epoch   1 |  2200/ 2928 batches | lr 5.00 | ms/batch 29.66 | loss  0.07 | ppl     1.07
| epoch   1 |  2400/ 